# Regular Expressions in Python

By far, the best course I have found on Regular Expressions is:  
https://www.safaribooksonline.com/videos/understanding-regular-expressions/9781491996300  

The best website I have found for learning and experimenting with Regular Expressions is:  
https://regex101.com/ 

## **re** module vs **regex** module

* **re** is part of standard library
* **regex** has latest PCRE with variable length look behind and more

In most cases, re and regex will do the same thing.  However with advanced regular expressions, you may find yourself wanting to use regex over re.

PCRE means Perl Compatible Regular Expressions

## re.search()

In practice, re.search() may be the method you use most.

re.search() will find the **first** match and return that as a match object.

If there is no match, it will return None.

The truthiness of None is bool(None) which is False.

In [1]:
# The last expression in a Jupyter Notebook cell is displayed using __repr__
# For None, nothing is displayed
None

In [2]:
# The truth value of None is False
bool(None)

False

In [3]:
# you can print(None) though
print(None)

None


In [4]:
import re

ml_text = """Machine learning is a method of data analysis that automates 
analytical model building. It is a branch of artificial intelligence 
based on the idea that systems can learn from data, identify patterns 
and make decisions with minimal human intervention.
"""

pattern = "minimal"
re.search(pattern, ml_text)

<re.Match object; span=(227, 234), match='minimal'>

In [5]:
# A Python pattern can be explictly compiled before usage
compiled_pattern = re.compile(pattern)
compiled_pattern.search(ml_text)

<re.Match object; span=(227, 234), match='minimal'>

In [6]:
# None is returned when there is no match, so nothing is displayed
pattern = "xyz"
re.search(pattern, ml_text)

## Compiling Regular Expressions
If you do not explicitly compile the pattern, Python will compile it for you.  It will also cache many of the patterns it compiles so it is does not have to recompile them when the same search is performed.

### Software Engineering Note
In most cases, optimizing code to make it run faster, makes it harder to read and maintain.  This incurs a software development cost that can be significant over the life cycle of the project.  As Knuth says, "premature optimization is the root of all evil".  It is usually best to write clear code.  Later the application can be profiled to see where time is being spent and that section of code can be optimized.  This approach reduces developer cost.

An exception to this is optimizing a nested loop from say O(n^2) to say O(nlog(n)) or faster.  Reducing the exponent of an operation that takes exponential time to run, will make a difference to overall application performance if the number of elements being processed is in the thousands or greater.  This is knowable without profiling.

In the case of regular expressions, the time being saved is just the compile time of the regular expression when it cannot be found in the cache or the lookup time to find it in the cache.  This is not much time.

However in this case, it can be argued that manually compiling the regular expression does not make the code harder to read and maintain. If it's not harder to read and maintain, then you might as well manually compile the regular expression, *outside of all loops*.

For this notebook, I will argue that manually compiling the regex is every so slightly harder to read and therefore it is not done to make learning easier.

## Jupyter HTML Cell Magic

The following allows for highlighting the part of the string that is matched.

In [7]:
from IPython.core.display import display, HTML
def highlight(match):
    """Display the matched part of the string with a different background color."""
    s = match.string
    start = match.start()
    end = match.end()
    span_start = '<span style="background-color:LightSalmon">'
    span_end = '</span>'
    html_string = s[:start] + span_start + s[start:end] + span_end + s[end:]
    display(HTML(html_string))

In [8]:
pattern = "minimal"
match = re.search(pattern, ml_text)
if match:
    highlight(match)

### re.search() Flags vs Regular Expression Flags

It is easier to read and maintain code using arguments to re.search() than to use flags inside the regular expression.

In rare situations, you may want the regex flag to be in effect for only part of the pattern.  To do this:
1. use the regex module instead of the re module
2. specify the flag inside of the regular expression at the location you want it to take effect

In [9]:
# using flags to re.search()
pattern = "machine"
match = re.search(pattern, ml_text, flags=re.IGNORECASE)
if match:
    highlight(match)

In [10]:
# same as above, but put the flag inside the regex itself
# this is harder to read!
pattern = "(?i)machine"
match = re.search(pattern, ml_text)
if match:
    highlight(match)

### Raw Strings

In Python, a raw string is string that is taken literally.

This is easiest to understand by example.

Usually a regular expression is expressed as a raw string in Python.

In [11]:
# \n represents new line
s = "a\nb"
print(s)

a
b


In [12]:
# \n is taken literally as the characters \ and n
s = r"a\nb"
print(s)

a\nb


## Quantifiers

```
* r'ab*'      # a followed by zero or more b's
* r'ab+'      # a followed by one or more b's
* r'ab?'      # a followed by zero or one b's
* r'ab{3}'    # a followed by three b's
* r'ab{2,3}'  # a followed by two to three b's
* r'ab{2,}'   # a followed by two or more b's
```

In [13]:
# a followed by 0+ b's
match = re.search(r'ab*', 'xax xabx ab aab')
if match:
    highlight(match)

In [14]:
# same as above, but find all matches
for match in re.finditer(r'ab*', 'xax xabx ab aab'):
    highlight(match)

In [15]:
# a followed by 0+ b's
# default is greedy: consume as many b's as possible
match = re.search(r'ab*', 'xabbx ab abbb')
if match:
    highlight(match)

In [16]:
# same as above, but find all matches
for match in re.finditer(r'ab*', 'xabbx ab abbb'):
    if match:
        highlight(match)

### Next Examples Will Use re.finditer()

As seen above, the first match with re.finditer() is the same as using re.search().  
Subsequent matches are only available to re.finditer()

In [17]:
# specify minimal match by placing '?' after quantifier
for match in re.finditer(r'ab*?', 'xabbx ab abbb'):
    if match:
        highlight(match)

In [18]:
# same as above but clearer
# minimal match of 0 or more b's is 0, so no need to have b at all
for match in re.finditer(r'a', 'xabbx ab abbb'):
    if match:
        highlight(match)

In [19]:
# a followed by 1+ b's
for match in re.finditer(r'ab+', 'xabbx ab abbb'):
    if match:
        highlight(match)

In [20]:
# a followed by 1+ b's, minimal match
for match in re.finditer(r'ab+?', 'xabbx ab abbb'):
    if match:
        highlight(match)

In [21]:
# same as above, but clearer
# minimal match of 1 or more b's is 1, so just use 1 b
for match in re.finditer(r'ab', 'xabbx ab abbb'):
    if match:
        highlight(match)

In [22]:
# a followed by 3 b's
for match in re.finditer(r'ab{3}', 'a ab abb abbbb abbb'):
    if match:
        highlight(match)

In [23]:
# a followed by 3 or more b's
for match in re.finditer(r'ab{3,}', 'a ab abb abbbbbb abbb'):
    if match:
        highlight(match)

## Anchors (aka Assertions)

In [24]:
# a followed by 2 b's AND
# assert preceeding character is non-word and following character is non-word
for match in re.finditer(r'\bab{2}\b', 'xabbx xabb abbx ab abbbbb abb abbbb abbb'):
    if match:
        highlight(match)

In [25]:
# space followed by a followed by 2 b's followed by space
# Note: this match includes the spaces on either side of abb
for match in re.finditer(r' ab{2} ', 'xabbx xabb abbx ab abbbbb abb abbbb abbb'):
    if match:
        highlight(match)

In [26]:
# a or b 2 or more times
for match in re.finditer(r'[a|b]{2,}', 'a xbax baab abb abbbb abbb'):
    if match:
        highlight(match)

In [27]:
# ab 2 or more times
for match in re.finditer(r'(ab){2,}', 'a ba baab xyabababz abbbb xababz'):
    if match:
        highlight(match)

In [28]:
# ab 2 or more times with non-word boundaries
for match in re.finditer(r'\b(ab){2,}\b', 'a ba abab baab ab xyabababz ababab xababz'):
    if match:
        highlight(match)

## Match any characters except ...

In [29]:
# match any character except '!' 1 or more times
match = re.search(r'[^!]+', '!Hello World!')
if match:
    highlight(match)

In [30]:
# match any character except '!' 1 or more times, minimal match
match = re.search(r'[^!]+?', '!Hello World!')
if match:
    highlight(match)

## Character Ranges

In [31]:
# match any lowercase character 1 or more times
for match in re.finditer(r'[a-z]+', 'Hello World!'):
    if match:
        highlight(match)

In [32]:
# match any uppercase character 1 or more times
for match in re.finditer(r'[A-Z]+', 'Hello World!'):
    if match:
        highlight(match)

## Sequences

In [33]:
# \w is shorthand for: [a-zA-Z0-9_]
# match any word character 1 or more times
for match in re.finditer(r'\w+', 'Hello World!'):
    if match:
        highlight(match)

In [34]:
# \d is shordhand for: [0-9]
# match any sequence of digits 1 or more times
match = re.search(r'\d+', 'Hello 123 World!')
if match:
    highlight(match)

In [35]:
# \D is shordhand for: [^0-9]
# match any sequence of nondigits 1 or more times
for match in re.finditer(r'\D+', 'Hello 123 World!'):
    if match:
        highlight(match)

In [36]:
# Match 1+ non-space characters
for match in re.finditer(r'\S+', 'This! is a string 1 2 345'):
    if match:                
        highlight(match)

In [37]:
# Match 1+ word characters followed by a space character
pattern = r'\w+ '
for match in re.finditer(r'\w+ ', 'This! is a string 1 2 345'):
    if match:
        highlight(match)

In [38]:
# Match 1+ word characters and
# assert next position is non-word character
pattern = r'\w+ '
for match in re.finditer(r'\w+\b', 'This! is a string 1 2 345'):
    if match:
        highlight(match)

In [39]:
# match any character any number of times, greedily, then match a digit
match = re.search(r'.*\d', 'This! is a string 1 2 345')
if match:
    highlight(match)

In [40]:
# match any character any number of times, minimually, then match a digit
match = re.search(r'.*?\d', 'This! is a string 1 2 345')
if match:
    highlight(match)

In [41]:
# match any non-word character one or more times
for match in re.finditer(r'\W+', 'This! is a string 1 2 345'):
    if match:
        highlight(match)

### RegEx Processing

At each position in the string, the regex is tried.  If it does not match, the next position in the string is tried.

The first match is always returned.

In [42]:
# try each alternative at each position in the string
s = 'An anteater encountered an ant on an antelope'

pattern = r'ant|antelope|anteater'
match = re.search(pattern, s)
if match:
    highlight(match)
    
pattern = r'anteater|antelope|ant'
match = re.search(pattern, s)
if match:
    highlight(match)
    
pattern = r'antelope|anteater|ant'
match = re.search(pattern, s)
if match:
    highlight(match)    

In [43]:
# try each alternative at each position in the string
# find all matches
s = 'An anteater encountered an ant on an antelope'

pattern = r'ant|antelope|anteater'
for match in re.finditer(pattern, s):
    if match:
        highlight(match) 

In [44]:
# try each alternative at each position in the string
# find all matches
s = 'An anteater encountered an ant on an antelope'

pattern = r'antelope|ant|anteater'
for match in re.finditer(pattern, s):
    if match:
        highlight(match) 

In [45]:
# try each alternative at each position in the string
# find all matches
s = 'An anteater encountered an ant on an antelope'

pattern = r'antelope|anteater|ant'
for match in re.finditer(pattern, s):
    if match:
        highlight(match) 

## Match RegEx Metacharacters

In [46]:
# Normally * and $ are special characters in a regex
# if you want to search for them, they need to be escaped
s = 'The 22nd item costs $999 or more'
pattern = r'\$\d+'

match = re.search(pattern, s)
if match:
    highlight(match)     

In [47]:
# $ as a metacharacter means match end of string
s = 'The 22nd item costs $999 or more'
pattern = r'\d+$'

# no match
match = re.search(pattern, s)
if match:
    highlight(match)

In [48]:
# $ as a metacharacter means match end of string
s = 'The 22nd item costs $999'
pattern = r'\d+$'

match = re.search(pattern, s)
if match:
    highlight(match)

In [49]:
# backtracking
# 1 or more alphanumeric characters, greedy, ending in 2 digits
for match in re.finditer(r'\w+\d\d', 'A123 BC3456 78 D7'):
    if match:
        highlight(match)

In [50]:
# backtracking
# 1 or more alphanumeric characters, minimal match, ending in 2 digits
for match in re.finditer(r'\w+?\d\d', 'A123 BC3456 78 D7'):
    if match:
        highlight(match)

In [51]:
# backtracking
# exactly 1 alphanumeric character, ending in 2 digits
for match in re.finditer(r'\w\d\d', 'A123 BC3456 78 D7'):
    if match:
        highlight(match)

In [52]:
# greedy search does not find what is inside each set of ""
pattern = r'".*"'
for match in re.finditer(r'".*"', '"Hello World!" and some "more" text "" and more'):
    if match:
        highlight(match)

In [53]:
# minimal search will find what is inside each set of ""
# Note: this search includes the double quotes
for match in re.finditer(r'".*?"', '"Hello World!" and some "more" text "" and more'):
    if match:
        highlight(match)

## Groups

In [54]:
def highlight_groups(match):
    """Display the matched part of each group, each with a different background color.
    
    This version will does not work if string has html chars such as '<' and '>'.
    """
    s = match.string
    num_groups = len(match.groups())
    
    if num_groups > 9:
        raise ValueError("Too Many Groups")
        
    colors = ['LightSalmon','GreenYellow', 'Orange'] * 3
    span_end = '</span>'
    
    # get start and end points
    starts = []
    ends = []
    for group_num in range(num_groups):
        start, end = match.span(group_num+1)
        starts.append(start)
        ends.append(end)
        
    starts.append(len(s))
        
    html_string = s[:starts[0]]
    for n in range(num_groups):
        span_start = f'<span style="background-color:{colors[n]}">'
        html_string = html_string + \
            span_start + s[starts[n]:ends[n]] + span_end + s[ends[n]:starts[n+1]]
    
    display(HTML(html_string))

In [55]:
# minimal search will find what is inside each set of ""
# note: this search does not include the double quotes
for match in re.finditer(r'"(.*?)"', '"Hello World!" and some "more" text "" and more'):
    if match:
        highlight_groups(match)

In [56]:
s = 'cp file1 file2 and some text mv oldfile newfile'
pattern = r'(?:cp|mv|ln)\s+(\w+)\s+(\w+)'
for match in re.finditer(pattern, s):
    if match:
        highlight_groups(match)

In [57]:
s = 'cp file1 file2 and some text mv oldfile newfile'
pattern = r'(cp|mv|ln)\s+(\w+)\s+(\w+)'
for match in re.finditer(pattern, s):
    if match:
        highlight_groups(match)

In [58]:
s = 'cp file1 file2 and some stuff mv oldfile newfile'
pattern = r'(?P<op>cp|mv|ln)\s+(?P<file1>\w+)\s+(?P<file2>\w+)'
for match in re.finditer(pattern, s):
    print(match.groupdict())

{'op': 'cp', 'file1': 'file1', 'file2': 'file2'}
{'op': 'mv', 'file1': 'oldfile', 'file2': 'newfile'}


In [59]:
s = """
This is string1
This is string2
"""
pattern = r'\d$'
print(re.findall(pattern,s))
print(re.findall(pattern,s,re.M))

['2']
['1', '2']


In [60]:
# negative (i.e. don't match) lookahead
s = 'end endwh1le endloop wh2le loop end'
pattern = r'end(?!\w)'
print(re.findall(pattern,s))

['end', 'end']


In [61]:
# positive (i.e. match) lookahead
pattern = r'end(?=\s)'
print(re.findall(pattern,s))

['end']


In [62]:
# positive (i.e. match) lookbehind
pattern = r'(?<!end)wh.le'
print(re.findall(pattern,s))

['wh2le']


In [63]:
pattern = r'\bwh.le\b'
print(re.findall(pattern,s))

['wh2le']


In [64]:
import regex
s = """
~123
~~1234
~~~12345
~~~~123456
"""
pattern = r'^~(?:~~)*\K\d+'
print(regex.findall(pattern,s,regex.M))

['123', '12345']


In [65]:
s = 'there are repeated repeated words in this string'
pattern = r'(\b\S+)\s+(\1)\b'
print(re.findall(pattern,s))

[('repeated', 'repeated')]


In [66]:
s = """
~123
~~1234
~~~12345
~~~~123456
"""
pattern = r"""
(?x)       # turn on extended formatting
^~(?:~~)*  # start of string (in multiline mode) followed by ~, followed by ~~
\K\d+      # restart matching followed by 1 or more digits
"""
print(regex.findall(pattern,s,regex.M))

['123', '12345']


In [67]:
pattern_compiled = re.compile(r"""(?x)
(put|get)\s+
(files|buffers)\s+
in\s+
(\w+)
"""
                             )

In [68]:
s = 'put files in mydirectory'
print(pattern_compiled.findall(s))
match = pattern_compiled.search(s)
print(match.groups())

[('put', 'files', 'mydirectory')]
('put', 'files', 'mydirectory')


In [69]:
pattern_compiled = re.compile(r"""(?x)
(?P<CMD>put|get)\s+
(?P<TYPE>files|buffers)\s+
in\s+
(?P<TARGET>\w+)
"""
                             )

In [70]:
s = 'put files in mydirectory'
print(pattern_compiled.findall(s))
match = pattern_compiled.search(s)
print(match.groups())

[('put', 'files', 'mydirectory')]
('put', 'files', 'mydirectory')


In [71]:
match.groupdict()

{'CMD': 'put', 'TYPE': 'files', 'TARGET': 'mydirectory'}

In [72]:
print(match.group("CMD"))
print(match.group("TYPE"))
print(match.group("TARGET"))

put
files
mydirectory


In [73]:
pattern_compiled = re.compile(r'(dog)')
match = pattern_compiled.fullmatch("dog")
match.groups()

('dog',)

In [74]:
s = 'dog'
pattern = r'(dog)'
pc = re.compile(pattern)
match = pc.search(s)
print(match)
print(match.groups())

<re.Match object; span=(0, 3), match='dog'>
('dog',)


In [75]:
print(match[0]) # all groups
print(match[1]) # group 1

dog
dog


### Regex Module

In [76]:
import regex
regex.DEFAULT_VERSION == regex.V0

True

In [77]:
s = 'abcdefg'
pattern = r'[[a-z]--[aeiou]](?V0)'
regex.findall(pattern, s)

[]

In [78]:
s = 'abcdefg'
pattern = r'[[a-z]--[aeiou]](?V1)'
regex.findall(pattern, s)

['b', 'c', 'd', 'f', 'g']

In [79]:
pattern = r'[[a-z]--[aeiou]]'

In [80]:
print(re.findall(pattern, s))
print(regex.findall(pattern, s))
print(regex.findall(pattern, s, regex.V0))
print(regex.findall(pattern, s, regex.V1))

[]
[]
[]
['b', 'c', 'd', 'f', 'g']


/home/agni/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Possible nested set at position 1
  """Entry point for launching an IPython kernel.


In [81]:
s = 'a-beautiful-day'

In [82]:
import sys
print(sys.version)

# new in 3.7, this now the same as with regex.split() below
re.split('(?=-)', s)

3.7.1 (default, Dec 14 2018, 19:28:38) 
[GCC 7.3.0]


['a', '-beautiful', '-day']

In [83]:
regex.split('(?=-)', s, regex.V1)

['a', '-beautiful', '-day']

In [84]:
whos

Variable           Type        Data/Info
----------------------------------------
HTML               type        <class 'IPython.core.display.HTML'>
compiled_pattern   Pattern     re.compile('minimal')
display            function    <function display at 0x7f9fe3880b70>
highlight          function    <function highlight at 0x7f9fdc8cf950>
highlight_groups   function    <function highlight_groups at 0x7f9fdc0376a8>
match              Match       <re.Match object; span=(0, 3), match='dog'>
ml_text            str         Machine learning is a met<...>mal human intervention.\n
pattern            str         [[a-z]--[aeiou]]
pattern_compiled   Pattern     re.compile('(dog)')
pc                 Pattern     re.compile('(dog)')
re                 module      <module 're' from '/home/<...>/ml/lib/python3.7/re.py'>
regex              module      <module 'regex' from '/ho<...>/site-packages/regex.py'>
s                  str         a-beautiful-day
sys                module      <module 'sys' (buil

In [85]:
regex.split('(?=-)', s)

['a', '-beautiful', '-day']

In [86]:
s = "<aa> b <ccde>"
r = r'<.*>'
mo = regex.match(r, s)
mo.captures()

['<aa> b <ccde>']

In [87]:
s = "<aa> b <ccde>"
r = r'<.*>'
mo = re.match(r, s)
mo.string

'<aa> b <ccde>'

In [88]:
s = "<aa> b <ccde>"
r = r'<.*?>'
mo = regex.match(r, s)
mo.captures()

['<aa>']

In [89]:
s = "<aa> b <ccde>"
r = r'<.*?>'
match_list = regex.findall(r, s)
match_list

['<aa>', '<ccde>']

In [90]:
s = "<aa> b <ccde>"
r = r'<.*>'
match_list = regex.findall(r, s)
match_list

['<aa> b <ccde>']

In [91]:
re.split(r'\W+', 'Words, words, and more words.')

['Words', 'words', 'and', 'more', 'words', '']

In [92]:
re.split(r'(?:\W+)', 'Words, words, and more words.')

['Words', 'words', 'and', 'more', 'words', '']

In [93]:
pattern = re.compile(r'(?P<digits>\d+) (?P<letters>[a-zA-Z])')
mo = pattern.search('123 45 6789 a234')
print(mo.group(0))
print(mo.group(1))
print(mo.group(2))
print(mo.groups())
print(mo.groupdict())

6789 a
6789
a
('6789', 'a')
{'digits': '6789', 'letters': 'a'}


In [94]:
pattern = re.compile(r'(?P<digits>\d+)')
mo = pattern.search('123 45 6789 a234')
print(mo.group(0))
print(mo.groups())
print(mo.groupdict())

123
('123',)
{'digits': '123'}


In [95]:
pattern = re.compile(r'(?P<digits>\d+)')
pattern.findall('123 45 6789 a234')

['123', '45', '6789', '234']

In [96]:
env = %env
env_list = [(key,value) for key,value in env.items()]
env_list.sort()
env_list[:5]

[('ADDR2LINE',
  '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-addr2line'),
 ('AR', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-ar'),
 ('AS', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-as'),
 ('CC', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-cc'),
 ('CFLAGS',
  '-march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe')]

In [97]:
print(type(env.keys()))

<class 'dict_keys'>


In [98]:
env = %env
keys = list(env.keys())
keys.sort()
for key in keys:
    m = regex.search(r'(?<=XDG)(\w+)', key)
    if m:
        print(m.group(1))

_CONFIG_DIRS
_CURRENT_DESKTOP
_DATA_DIRS
_MENU_PREFIX
_RUNTIME_DIR
_SEAT
_SESSION_DESKTOP
_SESSION_ID
_SESSION_TYPE
_VTNR


In [99]:
[k for k in keys if k.startswith('XDG')]

['XDG_CONFIG_DIRS',
 'XDG_CURRENT_DESKTOP',
 'XDG_DATA_DIRS',
 'XDG_MENU_PREFIX',
 'XDG_RUNTIME_DIR',
 'XDG_SEAT',
 'XDG_SESSION_DESKTOP',
 'XDG_SESSION_ID',
 'XDG_SESSION_TYPE',
 'XDG_VTNR']

In [100]:
[k for k in keys if regex.search(r'^XDG | ^PATH | ^LS_COLORS', k, flags=regex.X)]

['LS_COLORS',
 'PATH',
 'XDG_CONFIG_DIRS',
 'XDG_CURRENT_DESKTOP',
 'XDG_DATA_DIRS',
 'XDG_MENU_PREFIX',
 'XDG_RUNTIME_DIR',
 'XDG_SEAT',
 'XDG_SESSION_DESKTOP',
 'XDG_SESSION_ID',
 'XDG_SESSION_TYPE',
 'XDG_VTNR']

In [101]:
# env = %env
# env_list = [(key,value) for key,value in env.items() if not regex.search(r'^XDG | ^PATH | ^LS_COLORS | ^DBUS', key, flags=regex.X)]
# env_list.sort()
# env_list

In [102]:
# lsmagic
# pdoc regex

In [103]:
whos

Variable           Type        Data/Info
----------------------------------------
HTML               type        <class 'IPython.core.display.HTML'>
compiled_pattern   Pattern     re.compile('minimal')
display            function    <function display at 0x7f9fe3880b70>
env                dict        n=118
env_list           list        n=118
highlight          function    <function highlight at 0x7f9fdc8cf950>
highlight_groups   function    <function highlight_groups at 0x7f9fdc0376a8>
key                str         _CONDA_PYTHON_SYSCONFIGDATA_NAME
keys               list        n=118
m                  NoneType    None
match              Match       <re.Match object; span=(0, 3), match='dog'>
match_list         list        n=1
ml_text            str         Machine learning is a met<...>mal human intervention.\n
mo                 Match       <re.Match object; span=(0, 3), match='123'>
pattern            Pattern     re.compile('(?P<digits>\\d+)')
pattern_compiled   Pattern     re.comp

In [104]:
%psearch env*

env
env_list

In [105]:
timeit pass

6.05 ns ± 0.136 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [106]:
who

HTML	 compiled_pattern	 display	 env	 env_list	 highlight	 highlight_groups	 key	 keys	 
m	 match	 match_list	 ml_text	 mo	 pattern	 pattern_compiled	 pc	 r	 
re	 regex	 s	 sys	 


In [107]:
whos

Variable           Type        Data/Info
----------------------------------------
HTML               type        <class 'IPython.core.display.HTML'>
compiled_pattern   Pattern     re.compile('minimal')
display            function    <function display at 0x7f9fe3880b70>
env                dict        n=118
env_list           list        n=118
highlight          function    <function highlight at 0x7f9fdc8cf950>
highlight_groups   function    <function highlight_groups at 0x7f9fdc0376a8>
key                str         _CONDA_PYTHON_SYSCONFIGDATA_NAME
keys               list        n=118
m                  NoneType    None
match              Match       <re.Match object; span=(0, 3), match='dog'>
match_list         list        n=1
ml_text            str         Machine learning is a met<...>mal human intervention.\n
mo                 Match       <re.Match object; span=(0, 3), match='123'>
pattern            Pattern     re.compile('(?P<digits>\\d+)')
pattern_compiled   Pattern     re.comp

In [108]:
who_ls

['HTML',
 'compiled_pattern',
 'display',
 'env',
 'env_list',
 'highlight',
 'highlight_groups',
 'key',
 'keys',
 'm',
 'match',
 'match_list',
 'ml_text',
 'mo',
 'pattern',
 'pattern_compiled',
 'pc',
 'r',
 're',
 'regex',
 's',
 'sys']